In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

1.9.0


### Generate Torque Data and Labels 

In [68]:
# Parameter space restricting force vector lengths in y direction and defining length of door along x direction
x_min = 0
x_max = 10
y_min = -5
y_max = 5

def random_examples(N):
    """ Generate random force vector pairs
    
    Keyword arguments:
    N -- number of force vector pairs to produce. 

    """
    data = np.random.rand(N, 8) 
    data[:,0] *= x_max
    data[:,1] *= y_max
    data[:,2] *= x_max
    data[:,3] *= 0

    data[:,4] *= x_max
    data[:,5] *= y_min
    data[:,6] *= x_max
    data[:,7] *= 0
    
    return data

def perp_only_close(N):
    """ Generate random force vector pairs perpendicular to lever arm that result in 0 net torque
    
    Keyword arguments:
    N -- number of force vector pairs to produce. 

    """
    data = np.random.rand(N,8)
    data[:,0] *= x_max
    data[:,1] *= y_max
    data[:,2] = data[:,0]
    data[:,3] *= 0
    
    data[:,4] *= x_max
    data[:,5] = - data[:,0] * data[:,1] / data[:,4]
    data[:,6] = data[:,4]
    data[:,7] *= 0
    
    return data
    
def perp_only_open(N):
    """ Generate random force vector pairs perpendicular to lever arm that result in non-zero net torque
    
    Keyword arguments:
    N -- number of force vector pairs to produce. 

    """
    data = np.random.rand(N, 8) 
    data[:,0] *= x_max
    data[:,1] *= y_max
    data[:,2] *= data[:,0]
    data[:,3] *= 0

    data[:,4] *= x_max
    data[:,5] *= y_min
    data[:,6] = data[:,4]
    data[:,7] *= 0
    
    return data

def same_dist_close(N): 
    """ Generate random force vector pairs terminating at same point on lever arm that result in 0 net torque
    
    Keyword arguments:
    N -- number of force vector pairs to produce. 

    """
    data = np.random.rand(N, 8) 
    data[:,0] *= x_max
    data[:,1] *= y_max
    data[:,2] *= x_max
    data[:,3] *= 0

    data[:,4] *= x_max
    data[:,6] = data[:,2]
    data[:,5] = - data[:,2] * data[:,1] / data[:,6]
    data[:,7] *= 0
    
    return data

def same_dist_open(N):
    """ Generate random force vector pairs terminating at same point on lever arm that result in non-zero net torque
    
    Keyword arguments:
    N -- number of force vector pairs to produce. 

    """
    data = np.random.rand(N, 8) 
    data[:,0] *= x_max
    data[:,1] *= y_max
    data[:,2] *= x_max
    data[:,3] *= 0

    data[:,4] *= x_max
    data[:,5] *= y_max
    data[:,6] = data[:,2]
    data[:,7] *= 0
    
    return data
    
def same_mag_close(N): 
    pass

def same_mag_open(N):
    pass

N_rand = 50
N_perp_open = 50
N_perp_close = 50
N_dist_close = 50
N_dist_open = 50
N_mag_close = 10
N_mag_open = 10

N = N_rand + N_perp_open + N_perp_close + N_dist_close + N_dist_open + N_mag_close + N_mag_open

#TODO: Finish populating with data
data = np.vstack((random_examples(N_rand), perp_only_close(N_perp_close), perp_only_open(N_perp_open),
                    same_dist_close(N_dist_close), same_dist_open(N_dist_open))) 

labels = np.array([i[1] * i[2] + i[5] * i[6] == 0 for i in data]).astype(int)


### Preprocessing
- TODO: Normalize data for input into NN.

In [70]:
print(1 in labels)

True


### Create Model 
- TODO: Create model using NN architecture from tensorflow

In [71]:
print(data.shape)
print(labels.shape)

(250, 8)
(250,)


In [95]:
#TODO: Figure out optimal network architecture for this class of problem
model = keras.Sequential([
    # keras.layers.Flatten(input_shape=(1,8)),
    keras.layers.Dense(100, activation=tf.nn.relu),
    keras.layers.Dense(80, activation=tf.nn.relu),
    keras.layers.Dense(60, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [96]:
model.fit(data, labels, epochs=40)

Epoch 1/40
250/250 [==============================] - 1s 3ms/step - loss: 0.6437 - acc: 0.6160
Epoch 2/40
250/250 [==============================] - 0s 56us/step - loss: 0.5622 - acc: 0.6760
Epoch 3/40
250/250 [==============================] - 0s 61us/step - loss: 0.5380 - acc: 0.7080
Epoch 4/40
250/250 [==============================] - 0s 64us/step - loss: 0.5090 - acc: 0.7000
Epoch 5/40
250/250 [==============================] - 0s 64us/step - loss: 0.4774 - acc: 0.7360
Epoch 6/40
250/250 [==============================] - 0s 63us/step - loss: 0.4481 - acc: 0.7800
Epoch 7/40
250/250 [==============================] - 0s 64us/step - loss: 0.4249 - acc: 0.7840
Epoch 8/40
250/250 [==============================] - 0s 69us/step - loss: 0.4044 - acc: 0.7920
Epoch 9/40
250/250 [==============================] - 0s 62us/step - loss: 0.3877 - acc: 0.7960
Epoch 10/40
250/250 [==============================] - 0s 64us/step - loss: 0.3703 - acc: 0.8160
Epoch 11/40
250/250 [===================

### Small Data Model 
- Low complexity (high bias) model for small data
- Same number of layers as features at least (in this case, 8). 